# ONNX Safetensors Tutorial

This notebook demonstrates how to use the public API of the `onnx_safetensors` package to load and save ONNX models using safetensors.

In [ ]:
!pip install --upgrade onnx-safetensors onnx numpy

## Create a simple ONNX model

We create a simple ONNX model using ONNX helper APIs:

In [ ]:
import numpy as np
import onnx
from onnx import helper, TensorProto

# Create a simple model: Y = X + W, where W is an initializer
input_tensor = helper.make_tensor_value_info("input", TensorProto.FLOAT, [1, 3])
output_tensor = helper.make_tensor_value_info("output", TensorProto.FLOAT, [1, 3])

# Create an initializer tensor
weights = np.array([1.0, 2.0, 3.0], dtype=np.float32)
weights_initializer = helper.make_tensor(
    name="weights",
    data_type=TensorProto.FLOAT,
    dims=weights.shape,
    vals=weights.flatten().tolist(),
)

# Create a node (Add operation)
node_def = helper.make_node(
    "Add",
    inputs=["input", "weights"],
    outputs=["output"],
)

# Create the graph
graph_def = helper.make_graph(
    nodes=[node_def],
    name="SimpleGraph",
    inputs=[helper.make_tensor_value_info("input", TensorProto.FLOAT, [1, 3])],
    outputs=[helper.make_tensor_value_info("output", TensorProto.FLOAT, [1, 3])],
    initializer=[weights_initializer],
)

# Create the model
model_def = helper.make_model(graph_def, producer_name="onnx-safetensors-example")

# Save the model
onnx.save(model_def, "model.onnx")

## Loading tensors from a safetensors file into an ONNX model

We first create a safetensors file with compatible weights, then load these weights into the ONNX model.

In [ ]:
import onnx_safetensors
import numpy as np

# Create a safetensors file with compatible weights
weights_dict = {"weights": np.array([4.0, 5.0, 6.0], dtype=np.float32)}
onnx_safetensors.save(weights_dict, "weights.safetensors")

model = onnx.load("model.onnx")

onnx_safetensors.save_file(model, 'weights.safetensors', replace_data=False)

### Using safetensors as external data for ONNX

You can also save the ONNX model to use safetensors as external data.

In [ ]:
# Save the model to use safetensors as external data
model_with_external_data = onnx_safetensors.save_file(model, 'model.safetensors', base_dir='.', replace_data=True)

# Save the modified model
onnx.save(model, 'model_with_external_data.onnx')

Now, the ONNX model uses the safetensors file as external data, which can be loaded efficiently.